In [1]:
# Standard Libraries
import os # operating system dependent functionality
from collections import Counter # counting elements in an iterable

# External Libraries
import numpy as np # numerical operations on data arrays and matrices
import pandas as pd # data manipulation and analysis
import matplotlib.pyplot as plt # plotting and visualizations
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors

# Bioinformatics and Data Analysis 
import anndata # handling annotated data, particularly in genomics
import scanpy as sc # single-cell RNA-seq data analysis
import scipy # scientific and technical computations

# Test Specific Libraries
from sklearn.decomposition import FastICA
import cell2sentence as cs
from cell2sentence import tasks, CSData, CSModel
from cell2sentence.prompt_formatter import PromptFormatter
import ot
import cinemaot

# Huggingface
import torch
from transformers import AutoModelForCausalLM
from datasets import load_from_disk

In [2]:
# load processed AnnData objects for human and mouse cells
human_restricted = anndata.read_h5ad("/home/dor3/palmer_scratch/C2S_Files_Daphne/Cross_Species_Datasets/mouse_human_pancreas_tissue_Baron_et_al/processed_homolog_intersected_data/human_pancreas_one_sample_preprocessed_homolog_intersected_adata.h5ad")
mouse_restricted = anndata.read_h5ad("/home/dor3/palmer_scratch/C2S_Files_Daphne/Cross_Species_Datasets/mouse_human_pancreas_tissue_Baron_et_al/processed_homolog_intersected_data/mouse_pancreas_preprocessed_homolog_intersected_adata.h5ad")

In [3]:
# Ensure organism key is present for both human and mouse
if 'organism' not in human_restricted.obs:
    human_restricted.obs['organism'] = 'Homo sapiens'
if 'organism' not in mouse_restricted.obs:
    mouse_restricted.obs['organism'] = 'Mus musculus'

In [4]:
human_restricted

AnnData object with n_obs × n_vars = 1937 × 12113
    obs: 'cell_barcodes', 'cell_manual_ids', 'cell_types', 'batch_sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'organism'
    var: 'gene_name', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'ensembl_id'

Data processing

In [5]:
sc.pp.filter_cells(human_restricted, min_genes=200)
sc.pp.filter_genes(human_restricted, min_cells=3)

In [6]:
sc.pp.filter_cells(mouse_restricted, min_genes=200)
sc.pp.filter_genes(mouse_restricted, min_cells=3)

In [7]:
# determine effect of filtering
human_restricted

AnnData object with n_obs × n_vars = 1937 × 11601
    obs: 'cell_barcodes', 'cell_manual_ids', 'cell_types', 'batch_sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'organism'
    var: 'gene_name', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'ensembl_id'

In [8]:
# Count normalization
sc.pp.normalize_total(human_restricted)
# Lop1p transformation with base 10 - base 10 is important for C2S transformation!
sc.pp.log1p(human_restricted, base=10)  

In [9]:
# Count normalization
sc.pp.normalize_total(mouse_restricted)
# Lop1p transformation with base 10 - base 10 is important for C2S transformation!
sc.pp.log1p(mouse_restricted, base=10)  

Using Preprocessed data to embed cells

In [10]:
c2s_save_dir = "/home/sr2464/palmer_scratch/C2S_Files_Syed/c2s_api_testing"  # C2S dataset will be saved into this directory
human_c2s_save_name = "human_pancreas_tissue_c2s_embeddings_cinema_ot"  # This will be the name of our C2S dataset on disk
mouse_c2s_save_name = "mouse_pancreas_tissue_c2s_embeddings_cinema_ot"  # This will be the name of our C2S dataset on disk


In [11]:
arrow_dir='./data/'
# ensure the directory for Arrow datasets exists
os.makedirs(arrow_dir, exist_ok=True)

# paths to Arrow datasets
arrow_path_human = os.path.join(arrow_dir, 'human_arrow')
arrow_path_mouse = os.path.join(arrow_dir, 'mouse_arrow')

# convert AnnData to Arrow format and save
'''# Create CSData object
arrow_ds, vocabulary = cs.CSData.adata_to_arrow(
    adata=adata, 
    random_state=SEED, 
    sentence_delimiter=' ',
    label_col_names=adata_obs_cols_to_keep
)'''
human_arrow_ds, human_vocab = CSData.adata_to_arrow(
    adata=human_restricted, 
    random_state=1234, 
    sentence_delimiter=' ')

mouse_arrow_ds, mouse_vocab = CSData.adata_to_arrow(
    adata=mouse_restricted,
    random_state=1234, 
    sentence_delimiter=' ')

WARN: more variables (11601) than observations (1937)... did you mean to transpose the object (e.g. adata.T)?
WARN: more variables (11601) than observations (1937), did you mean to transpose the object (e.g. adata.T)?
100%|██████████| 1937/1937 [00:00<00:00, 3109.88it/s]
WARN: more variables (12113) than observations (1886)... did you mean to transpose the object (e.g. adata.T)?
WARN: more variables (12113) than observations (1886), did you mean to transpose the object (e.g. adata.T)?
100%|██████████| 1886/1886 [00:00<00:00, 3454.30it/s]


In [12]:
human_csdata = cs.CSData.csdata_from_arrow(
    arrow_dataset=human_arrow_ds, 
    vocabulary=human_vocab,
    save_dir=c2s_save_dir,
    save_name=human_c2s_save_name,
    dataset_backend="arrow"
)

Saving the dataset (0/1 shards):   0%|          | 0/1937 [00:00<?, ? examples/s]

In [13]:
mouse_csdata = cs.CSData.csdata_from_arrow(
    arrow_dataset=mouse_arrow_ds, 
    vocabulary=mouse_vocab,
    save_dir=c2s_save_dir,
    save_name=mouse_c2s_save_name,
    dataset_backend="arrow"
)

Saving the dataset (0/1 shards):   0%|          | 0/1886 [00:00<?, ? examples/s]

In [14]:
# Ensure the AnnData object has the same order of cells as the cell sentences list
human_cell_sentences = human_csdata.get_sentence_strings()
if len(human_cell_sentences) != human_restricted.shape[0]:
    raise ValueError("The number of cell sentences does not match the number of cells in the AnnData object.")

mouse_cell_sentences = mouse_csdata.get_sentence_strings()
if len(mouse_cell_sentences) != mouse_restricted.shape[0]:
    raise ValueError("The number of cell sentences does not match the number of cells in the AnnData object.")

In [15]:
# Add cell sentences to the AnnData object
human_restricted.obs['cell_sentence'] = human_cell_sentences

# Add cell sentences to the AnnData object
mouse_restricted.obs['cell_sentence'] = mouse_cell_sentences

In [16]:
# Optionally, print a few sentences to verify
print(human_restricted.obs[['cell_sentence']].head())

                                                                 cell_sentence
human1_lib1.final_cell_0001  REG1A-1 REG1A PRSS2-3 PRSS2 PRSS2-2 PRSS2-1 CE...
human1_lib1.final_cell_0002  REG1A-1 REG1A REG1B-1 REG1B PRSS2-1 PRSS2-2 PR...
human1_lib1.final_cell_0003  REG1A-1 REG1A PRSS2-1 PRSS2-3 PRSS2-2 PRSS2 RE...
human1_lib1.final_cell_0004  REG1A-1 REG1A REG1B-1 REG1B PRSS2-1 PRSS2-3 PR...
human1_lib1.final_cell_0005  REG1A-1 REG1A PRSS2-2 PRSS2-1 PRSS2-3 PRSS2 RE...


In [17]:
# Define CSModel object
cell_type_prediction_model_path = "/home/sr2464/palmer_scratch/C2S_Files_Syed/multicell_pretraining_v2_important_models/pythia-410m-multicell_v2_2024-07-28_13-55-51_checkpoint-7600_cell_type_pred"
save_dir = "/home/sr2464/palmer_scratch/C2S_Files_Syed/c2s_api_testing/csmodel_tutorial_2"
save_name = "cell_embedding_prediction_pythia_410M_1"
csmodel = cs.CSModel(
    model_name_or_path=cell_type_prediction_model_path,
    save_dir=save_dir,
    save_name=save_name)

Using device: cuda


In [18]:
# Embed cells
embedded_human_cells = tasks.embed_cells(
    csdata=human_csdata,
    csmodel=csmodel,
    n_genes=200,
)

Reloading model from path on disk: /home/sr2464/palmer_scratch/C2S_Files_Syed/c2s_api_testing/csmodel_tutorial_2/cell_embedding_prediction_pythia_410M_1


KeyError: 'organism'

In [ ]:
# Embed the cells using the C2S model
print("Embedding human cells...")
human_embeddings = tasks.embed_cells(csdata=human_csdata, csmodel=csmodel, n_genes=200)

Embedding human cells...
Reloading model from path on disk: /home/sr2464/palmer_scratch/C2S_Files_Syed/c2s_api_testing/csmodel_tutorial_2/cell_embedding_prediction_pythia_410M_1


KeyError: 'organism'

In [19]:
print("Embedding mouse cells...")
mouse_embeddings = tasks.embed_cells(csdata=mouse_csdata, csmodel=csmodel, n_genes=200, inference_batch_size=8)
print("Mouse cell embeddings shape: ", mouse_embeddings.shape)

Embedding mouse cells...
Reloading model from path on disk: /home/sr2464/palmer_scratch/C2S_Files_Syed/c2s_api_testing/csmodel_tutorial_2/cell_embedding_prediction_pythia_410M_1


KeyError: 'organism'

In [ ]:
# use ICA (Independence Component Analysis) to reduce the dimensionality of the embeddings
ica = FastICA(n_components=50) # What number should n_components be? I think for default PCA this is 14? Or is that nearest neighbors? Check.
human_ica = ica.fit_transform(human_embeddings)
mouse_ica = ica.fit_transform(mouse_embeddings) 

In [ ]:
# use CinemaOT to compute the cost matrix and OT plan
# create pandas dataframe to use with Cinema OT
human_df = pd.Dataframe(human_ica, index=adata_human.obs.index)
mouse_df = pd.Dataframe(mouse_ica, index=adata_mouse.obs.index)

In [ ]:
# use cinema OT to find optimal transport
ot_optimizer = cinemaot.OptimalTransport()
transport_plan = ot_op

In [ ]:
# extract OT matrix from the transport plan
ot_matrix = transport_plan.values


In [ ]:
 # find the ooptimal pairs based on the transport plan
paired_indices = []
for human_idx in range(ot_matrix.shape[0]):
    mouse_idx = np.argmax(ot_matrix[human_idx])
    paired_indices.append((human_idx, mouse_idx))

In [ ]:
def cinema_ot_pairing(adata_human, adata_mouse, csmodel, arrow_dir='./data/'):
    """
    Compute Cinema OT pairing between human and mouse cells using Cell2Sentence embeddings and ICA.

    Parameters:
    - adata_human: AnnData object for human cells.
    - adata_mouse: AnnData object for mouse cells.
    - csmodel: Cell2Sentence (CS) model for generating cell embeddings.

    Returns:
    - paired_indices: List of tuples (human_index, mouse_index) of paired cells
    - ot_matrix: Optimal transport matrix.
    """
    # ensure the directory for Arrow datasets exists
    os.makedirs(arrow_dir, exist_ok=True)
    
    # paths to Arrow datasets
    arrow_path_human = os.path.join(arrow_dir, 'human_arrow')
    arrow_path_mouse = os.path.join(arrow_dir, 'mouse_arrow')
    
    # prepare CSData objects
    csdata_human, vocab_human = prepare_csdata(adata_human, arrow_path_human)
    csdata_mouse, vocab_mouse = prepare_csdata(adata_mouse, arrow_path_mouse)

    # embed the cells using the C2S model (same space of c2s --> model's hidden configurations model.con)
    # human_embeddings.shape --> number of cells x hidden dimension (latent space size = hidden dimensionality of model)
    human_embeddings = tasks.embed_cells(csdata_human, csmodel, n_genes=200, inference_batch_size=8)
    mouse_embeddings = tasks.embed_cells(csdata_mouse, csmodel, n_genes=200, inference_batch_size=8)

    # use ICA (Independence Component Analysis) to reduce the dimensionality of the embeddings
    ica = FastICA(n_components=50) # What number should n_components be? I think for default PCA this is 14? Or is that nearest neighbors? Check.
    human_ica = ica.fit_transform(human_embeddings)
    mouse_ica = ica.fit_transform(mouse_embeddings) 

    # use CinemaOT to compute the cost matrix and OT plan
    # create pandas dataframe to use with Cinema OT
    human_df = pd.Dataframe(human_ica, index=adata_human.obs.index)
    mouse_df = pd.Dataframe(mouse_ica, index=adata_mouse.obs.index)

    # use cinema OT to find optimal transport
    ot_optimizer = cinemaot.OptimalTransport()
    transport_plan = ot_optimizer.optimize(human_df, mouse_df)

    # extract OT matrix from the transport plan
    ot_matrix = transport_plan.values

    # find the ooptimal pairs based on the transport plan
    paired_indices = []
    for human_idx in range(ot_matrix.shape[0]):
        mouse_idx = np.argmax(ot_matrix[human_idx])
        paired_indices.append((human_idx, mouse_idx))

    return paired_indices, ot_matrix

In [ ]:
paired_indices, ot_matrix = cinema_ot_pairing(human_restricted, mouse_restricted, CSModel)

WARN: more variables (12113) than observations (1937)... did you mean to transpose the object (e.g. adata.T)?
WARN: more variables (12113) than observations (1937), did you mean to transpose the object (e.g. adata.T)?
  0%|          | 0/1937 [00:00<?, ?it/s]


AttributeError: 'int' object has no attribute 'join'

In [ ]:
print(paired_indices[0])
print(paired_indices[1])
print(paired_indices[2])

In [ ]:
# tasks.embed_cells loads model from that path and then uses it

# Huggingface
'''import torch
from transformers import AutoModelForCausalLM
from datasets import load_from_disk

model = AutoModelForCausalLM.from_pretrained(
    csmodel.save_path,
    cache_dir=os.path.join(csmodel.save_dir, ".cache"),
    trust_remote_code=True
)
model = model.to(csmodel.device)'''

NameError: name 'csmodel' is not defined